In this activity, you are a data professional at an investment firm that is attempting to invest in private companies with a valuation of at least $1 billion. These are often known as "unicorns." Your client wants to develop a better understanding of unicorns, with the hope they can be early investors in future highly successful companies. They are particularly interested in the investment strategies of the three top unicorn investors: Sequoia Capital, Tiger Global Management, and Accel.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#to display all the columns
pd.set_option('display.max_columns', None)

In [3]:
companies = pd.read_csv('Modified_Unicorn_Companies.csv')
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


# DATA CLEANING

In [4]:
#display data types
companies.dtypes

,0
Company,object
Valuation,int64
Date Joined,object
Industry,object
City,object
Country/Region,object
Continent,object
Year Founded,int64
Funding,object
Select Investors,object


In [6]:
#change to specifi data types
companies['Date Joined'] = pd.to_datetime(companies['Date Joined'])

Add a column called Years To Unicorn, which is the number of years between when the company was founded and when it became a unicor

In [9]:
companies['Years To Unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']

**QUESTION: Why might your client be interested in how quickly a company achieved unicorn status?**

Learning how quickly a company achieves unicorn status may reveal certain trends or commonalities. Your client could leverage this information to find future companies to invest in.

Correcting bad data
Get descriptive statistics for the Years To Unicorn column.

In [10]:
companies['Years To Unicorn'].describe()

,Years To Unicorn
count,1074.000000
mean,7.013035
std,5.331842
min,-3.000000
25%,4.000000
50%,6.000000
75%,9.000000
max,98.000000


Isolate all rows where the Years To Unicorn column contains a negative value.

In [12]:
# Isolate any rows where `Years To Unicorn` is negative
companies[companies['Years To Unicorn'] < 0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Question: How many rows have negative values in the Years To Unicorn column, and what companies are they for?

There is a single row that has a negative value in the Years To Unicorn column. The company represented in this row is InVision.

An internet search reveals that InVision was founded in 2011. Replace the value at Year Founded with 2011 for InVision's row.

In [15]:
# Replace InVision's `Year Founded` value with 2011
companies.loc[companies['Company'] == 'InVision', 'Year Founded'] = 2011

companies[companies['Company'] == 'InVision']

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


Now, recalculate all the values in the Years To Unicorn column to remove the negative value for InVision. Verify that there are no more negative values afterwards.

In [16]:
companies['Years To Unicorn'] = companies['Date Joined'].dt.year - companies['Year Founded']
companies['Years To Unicorn'].describe()

,Years To Unicorn
count,1074.000000
mean,7.021415
std,5.323155
min,0.000000
25%,4.000000
50%,6.000000
75%,9.000000
max,98.000000


# Issues with Industry labels
 The company provided you with the following list of industry labels to identify in the data for Industry.

Note: Any labels in the Industry column that are not in industry_list are misspellings.

In [17]:
# List provided by the company of the expected industry labels in the data
industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',\
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',\
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation', \
        'Travel', 'Cybersecurity','Mobile & telecommunications']

In [18]:
# Check which values are in `Industry` but not in `industry_list`
set(companies['Industry']) - set(industry_list)


{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

# Question: Which values currently exist in the Industry column that are not in industry_list?

'Artificial Intelligence', 'Data management and analytics', and 'FinTech' are misspellings that are currently in the Industry column.

Now, correct the bad entries in the Industry column by replacing them with an approved string from industry_list. To do this, use the replace() Series method on the Industry series.

Example:

 [IN]: column_a = pd.Series(['A', 'B', 'C', 'D'])
       column_a

[OUT]: 0    A
       1    B
       2    C
       3    D
       dtype: object

 [IN]: replacement_dict = {'A':'z', 'B':'y', 'C':'x'}
       column_a = column_a.replace(replacement_dict)
       column_a

[OUT]: 0    z
       1    y
       2    x
       3    D
       dtype: object

Create a dictionary called replacement_dict whose keys are the incorrect spellings in the Industry series and whose values are the correct spelling, as indicated in industry_list.

Call the replace() method on the Industry series and pass to it replacement_dict as its argument. Reassign the result back to the Industry column.

Verify that there are no longer any elements in Industry that are not in industry_list.

In [20]:
#create replacement _dict
replacement_dict = {'Artificial Intelligence': 'Artificial intelligence',
                   'Data management and analytics': 'Data management & analytics',
                   'FinTech': 'Fintech'
                   }

#replace the incorrect values in the industry column
companies['Industry'] = companies['Industry'].replace(replacement_dict)

# 3. Verify that there are no longer any elements in `Industry` that are not in `industry_list`
set(companies['Industry']) - set(industry_list)

set()

Handling duplicate rows
The business mentioned that no company should appear in the data more than once.

Verify that this is indeed the case, and if not, clean the data so each company appears only once.

Begin by checking which, if any, companies are duplicated. Filter the data to return all occurrences of those duplicated companies.

In [21]:
#isolate rows that have duplicates
companies[companies.duplicated(subset = ['Company'] , keep = False)]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
385,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,United Kingdom,Europe,2007,$233M,"TSG Consumer Partners, Crowdcube",10
386,BrewDog,2,2017-04-10,Consumer & retail,Aberdeen,UnitedKingdom,Europe,2007,$233M,TSG Consumer Partners,10
510,ZocDoc,2,2015-08-20,Health,New York,United States,North America,2007,$374M,"Founders Fund, Khosla Ventures, Goldman Sachs",8
511,ZocDoc,2,2015-08-20,Health,NaN,United States,North America,2007,$374M,Founders Fund,8
1031,SoundHound,1,2018-05-03,Artificial intelligence,Santa Clara,United States,North America,2005,$215M,"Tencent Holdings, Walden Venture Capital, Glob...",13
1032,SoundHound,1,2018-05-03,Other,Santa Clara,United States,North America,2005,$215M,Tencent Holdings,13


# Question: Do these duplicated companies seem like legitimate data points, or are they problematic data?

The duplicated companies are not legitimate because they are clearly not different companies with the same name. They are the same company represented twice with minor variation.

In [ ]:
#drop dupes
companies = companies.drop_duplicates(subset='Company', keep='first')

# Question: Why is it important to perform input validation?

Input validation is an essential practice for ensuring data is complete, error-free, and high quality. A low-quality dataset may lend itself to an analysis that is incorrect or misleading.

**Question: What steps did you take to perform input validation for this dataset?**

The input validation steps for this lab included:

Fixing incorrect values

Correcting inconsistencies in the data

Removing duplicate data

**Convert numerical data to categorical data**

**Create a High Valuation column**

The data in the Valuation column represents how much money (in billions, USD) each company is valued at. Use the Valuation column to create a new column called High Valuation. For each company, the value in this column should be low if the company is in the bottom 50% of company valuations and high if the company is in the top 50%.

In [22]:
# Create new `High Valuation` column
# Use qcut to divide Valuation into 'high' and 'low' Valuation groups
companies['High Valuation'] = pd.qcut(companies['Valuation'], 2, labels=['low', 'high'])

**Convert categorical data to numerical data**

**Three common methods for changing categorical data to numerical are:**

Label encoding: order matters (ordinal numeric labels)

Label encoding: order doesn't matter (nominal numeric labels)

Dummy encoding: order doesn't matter (creation of binary columns for each possible category contained in the variable)

For example, a variable called subject might have possible values of history, mathematics, literature. In this case, each subject might be nominal—given the same level of importance. However, you might have another variable called class, whose possible values are freshman, sophomore, junior, senior. In this case, the class variable is ordinal—its values have an ordered, hierarchical structure of importance.
Machine learning models typically need all data to be numeric, and they generally use ordinal label encoding (method 1) and dummy encoding (method 3).

In the next steps, you'll convert the following variables: Continent, Country/Region, and Industry, each using a different approach.

# Convert Continent to numeric

**Question: Which type of variable would this make the Continent variable in terms of how it would be converted to a numeric data type?**

This would make Continent an ordinal variable, since more importance is placed on continents with fewer unicorn companies. There is a hierarchy of importance.

In [23]:
companies['Continent'].value_counts()

,count
Continent,
North America,588
Asia,310
Europe,144
South America,21
Oceania,8
Africa,3


Now, create a new column called Continent Number that represents the Continent column converted to numeric in the order of their number of unicorn companies, where North America is encoded as 1, through Africa, encoded as 6.

In [24]:
#create continent column
continent_dict =  {'North America': 1,
                  'Asia': 2,
                  'Europe': 3,
                  'South America': 4,
                  'Oceania': 5,
                  'Africa': 6}

companies['Continent Number'] = companies['Continent'].replace(continent_dict)
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3


**Convert Country/Region to numeric**
 suppose that within a given continent, each company's Country/Region is given equal importance. For analytical purposes, you want to convert the values in this column to numeric without creating a large number of dummy columns. Use label encoding of this nominal categorical variable to create a new column called Country/Region Numeric, wherein each unique Country/Region is assigned its own number

In [29]:
#create Country/Region Numeric column
companies['Country/Region Numeric'] = companies['Country/Region'].astype('category').cat.codes

**Convert Industry to numeric**
Finally, create dummy variables for the values in the Industry column.

In [30]:
# Convert `Industry` to numeric data

### YOUR CODE HERE ###

# Create dummy variables with Industry values
industry_encoded = pd.get_dummies(companies['Industry'])

# Combine `companies` DataFrame with new dummy Industry columns
companies = pd.concat([companies, industry_encoded], axis=1)

In [31]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn,High Valuation,Continent Number,Country/Region Numeric,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel,Artificial intelligence,Auto & transportation,Consumer & retail,Cybersecurity,Data management & analytics,E-commerce & direct-to-consumer,Edtech,Fintech,Hardware,Health,Internet software & services,Mobile & telecommunications,Other,"Supply chain, logistics, & delivery",Travel
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5,high,2,9,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,high,1,44,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10,high,2,9,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,Stripe,95,2014-01-23,Fintech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4,high,1,44,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6,high,3,38,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


**Question: Which categorical encoding approach did you use for each variable? Why?**

**Continent** - Ordinal label encoding was used because there was a hierarchical order to the categories.

**Country/Region** - Nominal label encoding was used because there was not a hierarchical order the categories.

**Industry** - Dummy encoding was used because there were not many different categories represented and they were all equally important.

**Question: How does label encoding change the data?**

Label encoding changes the data by assigning each category a unique number instead of a qualitative value.

**Question: What are the benefits of label encoding?**

Label encoding is useful in machine learning models, because many types of machine learning require all variables to be of a numeric data type.

**Question: What are the disadvantages of label encoding?**

Label encoding may make it more difficult to directly interpet what a column value represents. Further, it may introduce unintended relationships between the categorical data in a dataset.